In [1]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np

from tqdm import tqdm 

tqdm.pandas()

In [10]:
df_pred = pd.read_csv("../Data/DTW_df_pred.csv").drop("Unnamed: 0", axis=1)

In [11]:
df_pred.head()

,candidate_id,job_order,function_id,isco_code4
0,84556,0,1236.0,5131.0
1,84556,1,1065.0,9412.0
2,84556,2,1236.0,5131.0
3,84556,3,2038.0,9329.0
4,84556,4,2038.0,9329.0


In [4]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [5]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 468031/468031 [00:12<00:00, 38905.69it/s]


In [6]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1) & (career_lens <= 100)]

In [12]:
majority_class = df_pred["isco_code4"].value_counts().idxmax()

In [16]:
majority_class_performance = career_paths.apply(lambda x: x[-1] == majority_class).mean()

In [27]:
print(f"Majority class predictor accuracy: {majority_class_performance * 100:>.2f}%")

Majority class predictor accuracy: 14.55%
